In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome("S:\Projects/anime\ML\chromedriver\chromedriver_win32\chromedriver.exe")
wait = WebDriverWait(driver, 7)

C:\Users\mvspr\AppData\Local\Temp\ipykernel_46364\4269025399.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("S:\Projects/anime\ML\chromedriver\chromedriver_win32\chromedriver.exe")


In [2]:
from selenium.webdriver.common.by import By

links = []
score = []
driver.get('https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/stats')
link = driver.find_elements(By.CSS_SELECTOR, 'table[class="table-recently-updated"] td[class="borderClass"] a')
for item in link:
    links.append(item.get_attribute('href'))

In [3]:
len(links)

150

In [4]:
from selenium.webdriver.common.by import By

data = []
score = []
driver.get('https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/stats')
link = driver.find_elements(By.CSS_SELECTOR, 'table[class="table-recently-updated"] td[class="borderClass ac"]')
for item in link:
    data.append(item.text)

len(data)

300

In [5]:
len(data)

300

In [6]:
re_data = []
for i in range(0,300):
    if i%4 == 0:
        re_data.append([data[i],data[i+1],data[i+2],data[i+3]])
        
len(re_data)

75

In [7]:
up_links = []
for i in range(0,150):
    if (i%2 == 0):
        up_links.append(links[i])

len(up_links)

75

In [8]:
import pandas as pd

up_links = pd.DataFrame(up_links)
up_links.columns = ['user_link']
re_data = pd.DataFrame(re_data)
re_data.columns = ['score','status','Eps seen','activity']


full_data = pd.concat([up_links, re_data], axis=1, join='inner')
full_data.head(5)

,user_link,score,status,Eps seen,activity
0,https://myanimelist.net/profile/Northmeister,10,Completed,1 / 25,12 seconds ago
1,https://myanimelist.net/profile/liitledumpling,8,Completed,25 / 25,3 minutes ago
2,https://myanimelist.net/profile/AlexSzilagyi,-,Completed,25 / 25,3 minutes ago
3,https://myanimelist.net/profile/Michellelinchen,10,Completed,25 / 25,5 minutes ago
4,https://myanimelist.net/profile/eem_fahmeme,-,Completed,25 / 25,7 minutes ago


In [9]:
re_data.shape

(75, 4)

In [10]:
score_data = full_data.drop(['status','Eps seen', 'activity'], axis=1)
score_data.head(5)

,user_link,score
0,https://myanimelist.net/profile/Northmeister,10
1,https://myanimelist.net/profile/liitledumpling,8
2,https://myanimelist.net/profile/AlexSzilagyi,-
3,https://myanimelist.net/profile/Michellelinchen,10
4,https://myanimelist.net/profile/eem_fahmeme,-


In [11]:
anime_score = []
sampling = score_data.to_numpy()
for d in sampling:
    if(d[1].isdigit()):
        anime_score.append(d)

anime_score = pd.DataFrame(anime_score)

In [12]:
anime_score.columns = ['links','score']

In [13]:
anime_score['score'][0]

'10'

In [14]:
driver.get(anime_score['links'][0])

In [15]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

anime_stats = []
score = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div[class="stats anime"]')))
score = score.text
score = score.split('\n')
for s in score:
    s = s.split(' ')
    anime_stats.append(s)

anime_stats

[['Anime', 'Stats'],
 ['Days:', '22.5', 'Mean', 'Score:', '7.85'],
 ['Watching'],
 ['1'],
 ['Completed'],
 ['58'],
 ['On-Hold'],
 ['12'],
 ['Dropped'],
 ['3'],
 ['Plan', 'to', 'Watch'],
 ['87'],
 ['Total', 'Entries'],
 ['161'],
 ['Rewatched'],
 ['16'],
 ['Episodes'],
 ['1,344']]

In [17]:
driver.get('https://myanimelist.net/anime/16498/Shingeki_no_Kyojin/stats')
title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1[class="title-name h1_bold_none"]')))
title = title.text
print(title)

Shingeki no Kyojin


In [22]:
user_data = [0,title,anime_score['score'][0],anime_stats[1][1], anime_stats[1][4],anime_stats[3][0],anime_stats[5][0],anime_stats[7][0],anime_stats[9][0],anime_stats[11][0],anime_stats[13][0],anime_stats[15][0],anime_stats[17][0]]
for i in range(2,13):
    user_data[i] = user_data[i].replace(',','')
    user_data[i] = float(user_data[i])

u_data = []
for i in range(2):
    u_data.append(user_data)

u_data = pd.DataFrame(u_data)
u_data.columns = ['id','Title','Score','Days','Mean Score', 'Watching', 'Completed', 'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched', 'Episodes']
u_data = u_data.drop_duplicates()
u_data

,id,Title,Score,Days,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes
0,0,Shingeki no Kyojin,10.0,22.5,7.85,1.0,58.0,12.0,3.0,87.0,161.0,16.0,1344.0
